In [ ]:
import importlib, gffutils, re, HTSeq
import geneDrawingUtils, plot_interval
importlib.reload(plot_interval)


In [ ]:

"""
Building a gffutils database object.

Original document header:
    https://www.biostars.org/p/152517/
    Example of how to work with Ensembl release 81 GTF files, which:
        1) already have genes and transcripts included
        2) have unique IDs for genes, transcripts, and exons in the corresponding
           "<featuretype>_id" attribute
        3) do not have unique IDs for CDS, stop_codon, start_codon, UTR.
    See background info at on database IDs at:
        https://pythonhosted.org/gffutils/database-ids.html
    GTF file from
    ftp://ftp.ensembl.org/pub/release-81/gtf/mus_musculus/Mus_musculus.GRCm38.81.gtf.gz

For humans, the GTF file was obtained from:
http://ftp.ensembl.org/pub/release-104/gtf/homo_sapiens/Homo_sapiens.GRCh38.104.chr.gtf.gz   

"""

import gffutils, sys

def first_n_features(data, n=5000):
    """
    Useful for testing: only use the first `n` features of source data
    """
    for i, feature in enumerate(gffutils.iterators.DataIterator(data)):
        if i > n:
            break
        yield feature


# Note: this function is optional; if you don't want these IDs then comment out 
# the lines at [1] below
def subfeature_handler(f):
    """
    Given a gffutils.Feature object (which does not yet have its ID assigned),
    figure out what its ID should be.
    This is intended to be used for CDS, UTR, start_codon, and stop_codon
    features in the Ensembl release 81 GTF files.  I figured a reasonable
    unique ID would consist of the parent transcript and the feature type,
    followed by an autoincrementing number.
    See https://pythonhosted.org/gffutils/database-ids.html#id-spec for
    details and other options.
    """
    return ''.join(
        ['autoincrement:',
         f.attributes['transcript_id'][0],
         '_',
         f.featuretype])


# gffutils can spend a lot of time trying to decide on a unique ID for each
# feature. So we have to give it hints of where to look in the attributes.
#
# We also tell it to use our subfeature_handler function for featuretypes with
# no unique IDs.
id_spec = {
    'exon': 'exon_id',
    'gene': 'gene_id',
    'transcript': 'transcript_id',

    # [1] These aren't needed for speed, but they do give nicer IDs.
    'CDS': [subfeature_handler],
    'stop_codon': [subfeature_handler],
    'start_codon': [subfeature_handler],
    'UTR':  [subfeature_handler],
}



In [ ]:
# 1. Download a gtf of genome annotations.

# Download the annotation GTF and unzip:
# wget http://ftp.ensembl.org/pub/release-104/gtf/homo_sapiens/Homo_sapiens.GRCh38.104.chr.gtf.gz
# gunzip Homo_sapiens.GRCh38.104.chr.gtf.gz

# 2. Build the features.db object from the gtf.
gtf_filename = "Homo_sapiens.GRCh38.104.chr.gtf"
db = gffutils.create_db(
    gtf_filename,
    'assets/reference/features.no_introns.db',

    # Since Ensembl GTF files now come with genes and transcripts already in
    # the file, we don't want to spend the time to infer them (which we would
    # need to do in an on-spec GTF file)
    disable_infer_genes=True,
    disable_infer_transcripts=True,

    # Here's where we provide our custom id spec
    id_spec=id_spec,

    # "create_unique" runs a lot faster than "merge"
    # See https://pythonhosted.org/gffutils/database-ids.html#merge-strategy
    # for details.
    merge_strategy='create_unique',
    verbose=True,
    force=True,
)



In [ ]:
# 3. Create introns.
# This takes a long time.

intronDb = db.create_introns(exon_featuretype='exon', grandparent_featuretype='transcript')

# Using the recommended strategy of db.update() fails in my hands, so instead we use a work-around.
# We write a new gtf on just the introns.
with open('introns.gtf', 'w') as f:
    for li in intronDb:
        f.write(str(li) + '\n')

# Then concatenate the original gtf and the introns:
# cat Homo_sapiens.GRCh38.104.chr.gtf introns.gtf > Homo_sapiens.GRCh38.104.chr.with_introns.gtf

In [ ]:
# Create a new db, this time with introns.
gtf_filename = "Homo_sapiens.GRCh38.104.chr.with_introns.gtf"
db = gffutils.create_db(
    gtf_filename,
    'features.db',

    # Since Ensembl GTF files now come with genes and transcripts already in
    # the file, we don't want to spend the time to infer them (which we would
    # need to do in an on-spec GTF file)
    disable_infer_genes=True,
    disable_infer_transcripts=True,

    # Here's where we provide our custom id spec
    id_spec=id_spec,

    # "create_unique" runs a lot faster than "merge"
    # See https://pythonhosted.org/gffutils/database-ids.html#merge-strategy
    # for details.
    merge_strategy='create_unique',
    verbose=True,
    force=True,
)

# The genome annotation object is now complete!

In [ ]:
# 4. Load the input data.
import importlib, gffutils, re, HTSeq
import geneDrawingUtils, plot_interval
importlib.reload(plot_interval)

db = gffutils.FeatureDB('features.db', keep_order=True)
list(db.featuretypes())

In [ ]:
# Load a fasta if desired to write motif coverage.
genomic_fasta_fname = '/opt/genomes/gencode.v29/GRCh38.primary_assembly.genome.fa.gz'  # Newer, separate mapping.
genomic_fasta = dict( (s[1], s[0]) for s in HTSeq.FastaReader(genomic_fasta_fname, raw_iterator=True) )

In [ ]:
# 5. Plotting.
importlib.reload(plot_interval)
bw_fnames = {
    'Group1': ['data/Example_group1_rep1.bigwig', 'data/Example_group1_rep2.bigwig'],
    'Group2': ['data/Example_group2_rep1.bigwig', 'data/Example_group2_rep2.bigwig'],}


def igv_to_list(_str):
    chrom, rest = _str.split(':')
    rest = re.sub(',', '', rest)
    start, end = rest.split('-')
    return [chrom, int(start), int(end), '+']


motif = "[AT]GTGT[AT]"  # WGTGTW.
#iv = "chr4:151,097,347-151,106,361"  # RPS3aA
iv = "chr11:65,785,063-65,799,214"  # OVOL1
#iv = "chr14:24,247,114-24,265,177"  # TGM1
#iv = 'chr1:24,317,357-24,366,482'  # GRHL3
#iv = "chr17:41,752,607-41,788,768"  # JUP

clip_sites = [
    "chr17:41,752,607-41,788,768",  # JUP
    'chr1:24,317,357-24,366,482',  # GRHL3
    "chr14:24,247,114-24,265,177",  # TGM1
    "chr11:65,785,063-65,799,214",  # OVOL1
]
#iv = igv_to_list(iv)
_iv = [igv_to_list(x) for x in clip_sites]
for iv in _iv:
    plot_interval.plot_interval(iv, bw_fnames=bw_fnames, genomic_fasta=genomic_fasta, window=100, db=db, motif=motif, plot_replicates=False)